[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/ML_OSAE_M2/blob/ML_OSAE_M2/codes/CNN/ASIRRA_TensorFlow.ipynb)

In [ ]:
%%shell

cd /content/

wget https://share.obspm.fr/s/jqGotokdYDXTbDS/download/data_asirra.tar.gz

tar -xzf data_asirra.tar.gz

--2024-02-27 11:45:57--  https://share.obspm.fr/s/jqGotokdYDXTbDS/download/data_asirra.tar.gz
Resolving share.obspm.fr (share.obspm.fr)... 145.238.186.112
Connecting to share.obspm.fr (share.obspm.fr)|145.238.186.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 820207429 (782M) [application/gzip]
Saving to: ‘data_asirra.tar.gz’

data_asirra.tar.gz   95%[==================> ] 750.66M  26.5MB/s    eta 2s     

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from PIL import ImageOps
from tqdm import tqdm


def make_square(im, min_size=128, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im


def zoom_at(img, x, y, zoom):
    w, h = img.size
    zoom2 = zoom*2
    img = img.crop((x - w / zoom2, y - h / zoom2,
                    x + w / zoom2, y + h / zoom2))
    return img.resize((w, h), Image.LANCZOS)

def roll_zeropad(a, shift, axis=None):
    a = np.asanyarray(a)
    if shift == 0: return a
    if axis is None:
        n = a.size
        reshape = True
    else:
        n = a.shape[axis]
        reshape = False
    if np.abs(shift) > n:
        res = np.zeros_like(a)
    elif shift < 0:
        shift += n
        zeros = np.zeros_like(a.take(np.arange(n-shift), axis))
        res = np.concatenate((a.take(np.arange(n-shift,n), axis), zeros), axis)
    else:
        zeros = np.zeros_like(a.take(np.arange(n-shift,n), axis))
        res = np.concatenate((zeros, a.take(np.arange(n-shift), axis)), axis)
    if reshape:
        return res.reshape(a.shape)
    else:
        return res


def im_transform(im):
	im2 = im.copy()
	if(np.random.randint(0,2)):
		im2 = ImageOps.mirror(im2)
	im2 = im2.transform(im2.size, Image.AFFINE, (1.0-0.2*np.random.rand(),0.0, (np.random.rand()*2.0-1.0)*(im2.size[0]*0.1),
		0.0,1.0-0.2*np.random.rand(), (np.random.rand()*2.0-1.0)*(im2.size[1]*0.1)))
	#im2 = im2.rotate(np.random.rand()*40.0-20.0)
	im2 = im2.resize((128,128))

	return im2

orig_nb_images = 11500
test_size = 1000
image_size = 128
augm_fact = 2

train_im = np.zeros((orig_nb_images*augm_fact*2,image_size,image_size,3), dtype="uint8")

for i in tqdm(range(0, orig_nb_images)):
	im = Image.open("/content/data/PetImages/Cat/"+str(i)+".jpg")
	width, height = im.size

	im = make_square(im)
	width2, height2 = im.size

	x_offset = int((width2 - width)*0.5)
	y_offset = int((height2 - height)*0.5)

	im = im.resize((image_size,image_size))

	for k in range(0,augm_fact):
		im_array = np.asarray(im_transform(im))
		train_im[i*augm_fact+k,:,:,:] = im_array[:,:,:]

for i in tqdm(range(0,orig_nb_images)):
	im = Image.open("/content/data/PetImages/Dog/"+str(i)+".jpg")
	width, height = im.size

	im = make_square(im)
	width2, height2 = im.size

	x_offset = int((width2 - width)*0.5)
	y_offset = int((height2 - height)*0.5)

	im = im.resize((image_size,image_size))

	for k in range(0,augm_fact):
		im_array = np.asarray(im_transform(im))
		train_im[orig_nb_images*augm_fact+i*augm_fact+k,:,:,:] = im_array[:,:,:]

train_im.tofile("train_im.dat")

del (train_im)

test_im = np.zeros((test_size*2,image_size, image_size,3), dtype="uint8")

for i in tqdm(range(0, test_size)):
	im = Image.open("/content/data/PetImages/Cat/"+str(orig_nb_images+i)+".jpg")
	width, height = im.size

	im = make_square(im)
	width2, height2 = im.size

	x_offset = int((width2 - width)*0.5)
	y_offset = int((height2 - height)*0.5)

	im = im.resize((image_size,image_size))

	im_array = np.asarray((im))
	test_im[i,:,:,:] = im_array[:,:,:]

for i in tqdm(range(0,test_size)):
	im = Image.open("/content/data/PetImages/Dog/"+str(orig_nb_images+i)+".jpg")
	width, height = im.size

	im = make_square(im)
	width2, height2 = im.size

	x_offset = int((width2 - width)*0.5)
	y_offset = int((height2 - height)*0.5)

	im = im.resize((image_size,image_size))

	im_array = np.asarray((im))
	test_im[test_size+i,:, :,:] = im_array[:,:,:]


test_im.tofile("test_im.dat")

del (test_im)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageOps

import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics
import tensorflow as tf


image_size = 128
nb_per_class = 11500
nb_test = 1000
augm_fact = 2
train_data = np.fromfile("train_im.dat", dtype="uint8")
train_data = np.reshape(train_data, ((nb_per_class*2*augm_fact,image_size,image_size,3)))

train_targets = np.zeros((nb_per_class*2*augm_fact,2), dtype="float32")

#train_data[:] /= 255.0

train_targets[0:nb_per_class*augm_fact,0] = 1.0
train_targets[nb_per_class*augm_fact:,1]  = 1.0

test_data = np.fromfile("test_im.dat", dtype="uint8")
test_data = np.reshape(test_data, ((nb_test*2,image_size,image_size,3)))

test_targets = np.zeros((nb_test*2,2), dtype="float32")

#test_data[:] /= 255.0

test_targets[0:nb_test,0] = 1.0
test_targets[nb_test:,1]  = 1.0



In [ ]:

######################### ##########################
#         Loading the neural network model
######################### ##########################

model = keras.Sequential()

model.add(layers.ZeroPadding2D(padding=(1, 1), input_shape=(image_size,image_size,3)))
model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D())

model.add(layers.ZeroPadding2D(padding=(1, 1)))
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D())

model.add(layers.ZeroPadding2D(padding=(1, 1)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D())

model.add(layers.ZeroPadding2D(padding=(1, 1)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=2, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


######################### ##########################
#                 Network training
######################### ##########################

print(model.summary())

model.fit(train_data, train_targets, batch_size=32, epochs=40, shuffle=True,  validation_split=0.0, validation_data=(test_data, test_targets))

######################### ##########################
#            Evaluate the network prediction
######################### ##########################

model.evaluate(test_data, test_targets)

pred = model.predict(test_data)

matrix = metrics.confusion_matrix(test_targets.argmax(axis=1), pred.argmax(axis=1))

print (matrix)


